# STC Jawwy

In [1]:
!pip install pyxlsb

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# Import the required libraries 

import pandas as pd     # provides high-performance, easy to use structures and data analysis tools
import pyxlsb           # Excel extention to read xlsb files (the input file)
import numpy as np      # provides fast mathematical computation on arrays and matrices

# Jawwy dataset
The dataset consists of meta details about the movies and tv shows as genre. 
Also details about Users activities, spent duration and if watching in High definition or standard definition. 
You have to analyse this dataset to find top insights, findings and to solve the four tasks assigned to you.

In [2]:
dataframe = pd.read_excel("stc TV Data Set_T1.xlsb",sheet_name="Final_Dataset", engine = 'pyxlsb')

In [44]:
# check the data shape
dataframe.shape

(1048575, 12)

In [45]:
# display the first 5 rows 
dataframe.head()

,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name
0,2017-05-27,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets
1,2017-05-21,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana
2,2017-08-10,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
3,2017-07-26,19307,The Mermaid Princess,76,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
4,2017-07-07,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,1,Churchill


In [3]:
# Data Preprocessing on the input data
dataframe = dataframe.drop(columns=['Column1'])         # dropping the index column
dataframe['program_name'] = dataframe['program_name'].str.strip()  # trim spaces in movies names to avoid misspellings in input data
dataframe['date_'] = pd.to_datetime(dataframe['date_'], unit='d', origin='30/12/1899')  # read date column as date data type
dataframe[['duration_seconds', 'season','episode','series_title','hd']] = dataframe[['duration_seconds', 'season','episode','series_title','hd']].apply(pd.to_numeric)  # read numeric columns as numeric data types
dataframe[['user_id_maped', 'program_name','program_class','program_desc','program_genre','original_name']] = dataframe[['user_id_maped', 'program_name','program_class','program_desc','program_genre','original_name']].astype(str) # read string columns as string data types

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: UserWarning: Parsing '30/12/1899' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# display the dataset after applying data types
dataframe.dtypes

date_               datetime64[ns]
user_id_maped               object
program_name                object
duration_seconds             int64
program_class               object
season                       int64
episode                      int64
program_desc                object
program_genre               object
series_title                 int64
hd                           int64
original_name               object
dtype: object

In [5]:
# describe the numeric values in the dataset
dataframe.describe()

,duration_seconds,season,episode,series_title,hd
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,1.230957e+03,1.342139e+00,6.157952e+00,1.205922e-02,3.862728e-01
std,6.821058e+03,2.104095e+00,1.222015e+01,1.091504e-01,4.868946e-01
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.190000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,1.328000e+03,1.000000e+00,9.000000e+00,0.000000e+00,1.000000e+00
max,1.461329e+06,2.300000e+01,2.820000e+02,1.000000e+00,1.000000e+00


In [6]:
# check if any column has null value in the dataset
dataframe.isnull().any()

date_               False
user_id_maped       False
program_name        False
duration_seconds    False
program_class       False
season              False
episode             False
program_desc        False
program_genre       False
series_title        False
hd                  False
original_name       False
dtype: bool

# Task 1
###### get the most watched movies (Total Views / Total Users Views / Total watch time)


In [9]:
# make a copy of the dataframe for working on task 1
df=dataframe.copy()

In [10]:
df.dtypes

date_               datetime64[ns]
user_id_maped               object
program_name                object
duration_seconds             int64
program_class               object
season                       int64
episode                      int64
program_desc                object
program_genre               object
series_title                 int64
hd                           int64
original_name               object
dtype: object

In [41]:
# Here we try to get the most watched movies (Total Views / Total Users Views / Total watch time)
# For series we concatenated the Session episode to differentiate between episodes 
groups=df.copy()
groups.loc[groups['program_class'] == 'SERIES/EPISODES', 'program_name'] = groups['program_name']+'_SE'+groups['season'].astype(str)+'_EP'+groups['episode'].astype(str)  ##modify the program name column by appeanding a string that includes season and episode info, but only for rows where the program class column has the value series/episodes.


groups = groups.groupby(['program_name','program_class'])\
.agg({'user_id_maped': [('co11', 'nunique'),('co22', 'count')],\
      'duration_seconds': [('co33', 'sum')] }).reset_index() # calculate number of unique values and total count of value, in addition to sum of total time watched
groups.columns = ['program_name','program_class','No of Users who Watched', 'No of watches', 'Total watch time in seconds']
groups['Total watch time in hours']=groups['Total watch time in seconds']/3600
groups = groups.drop(columns=['Total watch time in seconds'])
groups = groups.sort_values(by=['Total watch time in hours', 'No of watches','No of Users who Watched'], ascending=False).reset_index(drop=True)


In [43]:
# show the result
groups.head()

,program_name,program_class,No of Users who Watched,No of watches,Total watch time in hours
0,The Boss Baby,MOVIE,3389,24047,2961.350833
1,The Amazing pider-Man,MOVIE,1011,2877,1966.119167
2,The Expendables,MOVIE,853,2119,1961.159444
3,Moana,MOVIE,2173,8081,1706.176944
4,Trolls,MOVIE,2613,13793,1601.023056


In [39]:
import matplotlib.pyplot as plt 
import plotly
import plotly.express as px
import plotly.offline as pyo 
pyo.init_notebook_mode(connected = True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [44]:
# plot top 10 Programs 
fig = px.pie(groups.head(10), values='Total watch time in hours', names='program_name',\
             hover_data=['program_class'],title='Top 10 programs in total watch time in hours')
fig.update_traces(sort=True)
fig.show()

This chart demonestrated the most 10 watched movies.

In [15]:
# get the top 10 most watched sereis and see whether viewers watched in hd or sd.

top_ten_series = groups[groups['program_class'] == 'SERIES/EPISODES'].nlargest(10, 'Total watch time in hours')

In [16]:
top_ten_series.head()

,program_name,program_class,No of Users who Watched,No of watches,Total watch time in hours
31,Blindspot Erase Weary Youth_SE1_EP13,SERIES/EPISODES,125,197,516.015556
36,PAW Patrol Pups Make A plash / Pups Fall Fest...,SERIES/EPISODES,407,1774,485.739722
37,PAW Patrol Pups ave The ea Turtles_SE1_EP2,SERIES/EPISODES,334,1439,478.470278
42,PAW Patrol Pups And The Kitty-Tastrophe_SE1_EP3,SERIES/EPISODES,294,1296,455.909722
45,PAW Patrol Pup Pup Goose / Pup Pup And Away_S...,SERIES/EPISODES,268,1222,448.070000


In [40]:
# plot top 10 series
# group by series
top_ten_series['hd'] = df['hd']

fig = px.pie(top_ten_series, values = 'Total watch time in hours', names = 'program_name',\
            hover_data = ['hd'],title = 'Top 10 series in total watch time in hours')
fig.update_traces(sort=True)
fig.show()

In [17]:
# Here we try to study the customer experience against Program class
grouped=df.copy()
grouped = grouped.groupby('program_class')\
.agg({'user_id_maped': [('co1', 'nunique'),('co2', 'count')],\
      'duration_seconds': [('co3', 'sum')] }).reset_index()
grouped.columns = ['program_class','No of Users who Watched', 'No of watches', 'Total watch time in seconds']
grouped['Total watch time in hours']=grouped['Total watch time in seconds']/3600
grouped = grouped.drop(columns=['Total watch time in seconds'])
grouped = grouped.sort_values(by=['Total watch time in hours', 'No of watches','No of Users who Watched'], ascending=False).reset_index(drop=True)


In [18]:
# show the result
grouped.head()

,program_class,No of Users who Watched,No of watches,Total watch time in hours
0,SERIES/EPISODES,3901,560174,255097.787500
1,MOVIE,11355,488401,103444.145556


In [19]:
# plot the total watch time against total number of users and report your findings
fig = px.pie(grouped, values='Total watch time in hours', names='program_class',\
             hover_data=['program_class'],title='Total duration spent by program_class')
fig2 = px.pie(grouped, values='No of Users who Watched', names='program_class',\
             hover_data=['program_class'],title='Total Users watching by program_class')

fig.update_traces(sort=False)
fig2.update_traces(sort=False)
fig.show()
fig2.show()

#### Analysis:
Most users spend more time watching series, this might be due to series are combined of multiple episodes and seasons of multiple hours.

74.4% of users watch movies and 25.6% watch series. This might be up to the fact that series are longer and take longer time to complete compared to movies which are a one-take to complete.

-------------------------------------------------

In [21]:

#study the relation and user's behaviour Against HD flag
grouped=df.copy()
grouped = grouped.groupby('hd')\
.agg({'user_id_maped': [('co1', 'nunique'),('co2', 'count')],\
      'duration_seconds': [('co3', 'sum')] }).reset_index()
grouped.columns = ['HD viewership','No of Users who Watched', 'No of watches', 'Total watch time in seconds']
grouped['Total watch time in hours']=grouped['Total watch time in seconds']/3600
grouped = grouped.drop(columns=['Total watch time in seconds'])
grouped = grouped.sort_values(by=['Total watch time in hours', 'No of watches','No of Users who Watched'], ascending=False).reset_index(drop=True)


In [22]:
grouped.head()

,HD viewership,No of Users who Watched,No of watches,Total watch time in hours
0,0,6728,643539,268364.372778
1,1,11000,405036,90177.560278


In [23]:
# visualize the results using clusterd column chart

fig = go.Figure()
for behavior in grouped[1:]:
    fig.add_trace(go.Bar(
                        name = behavior,
                        x=grouped.index,
                        y = grouped[behavior])
                        )
    
# set the chart title and axis lavels
fig.update_layout(
            title = ' User behaviors by HD Category',
            xaxis = dict(title = 'HD'),
            yaxis = dict(title = 'Aggregation'),
            barmode = 'group')

fig.show()

#### *******Analysis: 
on average, most users watch in SD, whether be it movies or series. They also spend more time watching in SD quality. This might be attributable to the fact most of the media presented are in the SD quality, it also might suggest that users are saving their data.


----------------

In [24]:
# study the relation between HD viewership and program class

group = df.copy()

group = group.groupby(['program_class', 'hd'])\
            .agg({'user_id_maped': [('co1', 'nunique'),('co2', 'count')],\
            'duration_seconds': [('co3', 'sum')] }).reset_index()

group.columns = ['program_class','HD viewership','No of Users who Watched', 'No of watches', 'Total watch time in seconds']
group['Total watch time in hours']=group['Total watch time in seconds']/3600
group = group.drop(columns=['Total watch time in seconds'])
group = group.sort_values(by=['Total watch time in hours', 'No of watches','No of Users who Watched'], ascending=False).reset_index(drop=True)


In [25]:
group.head()


,program_class,HD viewership,No of Users who Watched,No of watches,Total watch time in hours
0,SERIES/EPISODES,0,3282,486884,229776.593333
1,MOVIE,1,10880,331746,64856.366111
2,MOVIE,0,6093,156655,38587.779444
3,SERIES/EPISODES,1,2625,73290,25321.194167


#### Analysis:
mostly, users prefer to watch series in SD quality, as oppose to movies which they prefer to watch in HD quality. This might assist STC TV producers to promote SD viewership or perhaps get series in HD format if they were'nt available in HD.


----

In [26]:
# analyze if there's a relation between the date, or specifically month, and the amount of users who watched.
groups = dataframe.copy()
groups['month'] = groups['date_'].dt.month
groups = groups.groupby('month').agg({'user_id_maped': ('nunique')}).reset_index()

groups.columns = ['month', 'Number of users who watched']
trace = go.Scatter(name = 'month', x = groups['month'], y = groups['Number of users who watched'])
layout = go.Layout(
                title = 'Number of users per each month',
                xaxis = dict(title = 'Month'),
                yaxis = dict(title = 'Number of users'))
fig = go.Figure(data = [trace], layout = layout)

# set the chart title and axis lavels

fig.show()

#### ***Analysis:
most watches are in the first quarter of the year. This viewership trend analysis helps to identify peak months to optimize content scheduling.

In [18]:
##! shell escape syntax to run shell commands directly into the notebook
#!jupyter nbconvert --to pdf --pandoc C:\Users\sh\AppData\Roaming\Pandoc stc_TV_T1_4.ipynb

In [27]:
# most wathced genere
df = dataframe.copy()
genre = df['program_genre'].value_counts()
genre = pd.DataFrame(genre).reset_index()

In [28]:
# animation is the most wathced genre, so it the team identify what most genre is used to optimize the view quality, and to present more of what the audience prefer.

In [29]:
genre.head()


,index,program_genre
0,Animation,401730
1,Action,174155
2,Drama,155048
3,Comedy,136577
4,Horror,75306


In [30]:
genre.columns = ['program_genre', 'No of watches']
genre.values

array([['Animation', 401730],
       ['Action', 174155],
       ['Drama', 155048],
       ['Comedy', 136577],
       ['Horror', 75306],
       ['Thriller', 39601],
       ['Family', 26668],
       ['Biography', 16135],
       ['Documentary', 13291],
       ['NOT_DEFINED_IN_UMS', 4535],
       ['Crime', 3232],
       ['Sci-Fi', 1322],
       ['Romance', 512],
       ['Adventure', 239],
       ['SERIES_NOT_ADDED_UNDER_ANY_GENRE', 216],
       ['Wrestling', 8]], dtype=object)

In [31]:
#plot the most wathced genres

trace = px.bar(
              x = genre['program_genre'],
              y = genre['No of watches'])


fig = go.Figure(data = trace)
fig.show()

#### Analysis:
this graph shows the preference of users to watch Animation, second by Action and Drame genres.
So it helps the team identify what most genre is watched to optimize the view quality, and to present more of what the audience prefer.